In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!ls

 cached_dev_trob55007534best_384
 Classroom
'Colab Notebooks'
 ColabNotebooks
 eqa
 eqa.rar
'Google Summer of Code Project Proposal EduAid.gdoc'
 Training-a-Model-for-Neural-Question-Answering-over-DBpedia-Ridhiman-Kaur-Dhindsa.gdoc


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.9 MB/s eta 0:00:00


In [5]:
import json   
import os     
import torch  
input_file='./eqa/predict.json' 
from transformers import (
    MODEL_FOR_QUESTION_ANSWERING_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    get_linear_schedule_with_warmup) 
model = AutoModelForQuestionAnswering.from_pretrained("./eqa/trob55007534best") 
tokenizer = AutoTokenizer.from_pretrained("./eqa/trob55007534best")  
f=open(input_file,"r")    
input_data = json.load(f)
qa_map = [] 
question_number=1 
for entry in input_data['data']:  
    paragraph_text = entry['context'] 
    print("Context:",paragraph_text) 
    for question_text in entry['questions']:     
        print("Question:",question_number,":",question_text) 
        question_number=question_number+1     
        inputs = tokenizer(question_text, paragraph_text, add_special_tokens=True, return_tensors="pt") 
        input_ids = inputs["input_ids"].tolist()[0]  
        text_tokens = tokenizer.convert_ids_to_tokens(input_ids) 
        answer_start_scores, answer_end_scores = model(**inputs,return_dict=False) 
        answer_start = torch.argmax(answer_start_scores)  
        answer_end = torch.argmax(answer_end_scores) + 1              
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))  
        
        print("Answer: ", answer)      
        qa_map.append(question_text) 
        qa_map.append(answer)                                                               
result_json = json.dumps(qa_map)     
os.makedirs('./eqa/results', exist_ok=True)
with open('./eqa/results/answers.json', 'w') as f:  
    json.dump(result_json, f)        
print("Prediction Completed")

Context: Superintendence, direction and control of Parliamentary elections as well as Assembly elections of the States/UTs is vested in the Election Commission of India(ECI) by virtue of Article 324 of the Constitution of India, hence it is obligatory and incumbent on the ECI to ensure that each and every election is conducted in a free, fair, transparent and peaceful manner. It has been the endeavour of the ECI that the level playing field for all stake holders, including candidates and political parties is not disturbed and the electoral process is not allowed to be vitiated by any means including misuse of money power. The Election Commission vowed to respond to the emerging challenges in overall election management, faced with increasing threat from those who were hell bent on distorting the mandate of the electors and as such adopted a structured mechanism to facilitate and monitor election expenditure starting with the elections to the Legislative Assembly of Bihar held in 2010. 

In [ ]:
!pip freeze >> ./eqa/requirements.txt